In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',0)
# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for variable transformation
import scipy.stats as stats

import os
import gc
from tqdm import tqdm_notebook
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from scipy.optimize import differential_evolution
from sklearn.model_selection import GridSearchCV  
from scipy.optimize import differential_evolution
from datetime import datetime
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score
from scikitplot.estimators import plot_feature_importances
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm
import sklearn.metrics as metrics
from datetime import datetime
import re
seed = 42

In [2]:
def parse_month(df,cols,newcol=False):
    for col in cols:
        if newcol:
            cn = col+'_parsed' 
        else: cn = col
        df[cn] = pd.to_datetime(df[col],format='%Y%m')
    return df

In [3]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
          # Total valores perdidos
        mis_val = df.isnull().sum()
         # porcentaje de valores perdidos
        mis_val_percent = 100 * df.isnull().sum() / len(df)
         # crea una tabla con los resultados
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # renombra columnas
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'valores perdidos', 1 : '% de valores totales'})
       # ordena de mayor a menor los porcentaje de valores perdidos
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% de valores totales', ascending=False).round(1)
        # te muestra la informacion resumida
        print ("la base tiene " + str(df.shape[1]) + " columns.\n"      
            "estos son " + str(mis_val_table_ren_columns.shape[0]) +
              " columnas que tienen valores perdidos.")
         # retorna a un dataframe la informacion
        return mis_val_table_ren_columns

In [4]:
pd.set_option('display.max_colwidth',1000)

In [5]:
file_path = 'D:/usil/'

In [6]:
target_202201_202203 = pd.read_csv(file_path+'08_target_202201_202203.csv')#
convergente_202201_202204 = pd.read_csv(file_path+'07_convergente_202201_202204.csv')
trafico_202201_202204 = pd.read_csv(file_path+'06_trafico_202201_202204.csv')#
terminales_202201_202204 = pd.read_csv(file_path+'05_terminales_202201_202204.csv')#
roaming_202201_202204 = pd.read_csv(file_path+'04_roaming_202201_202204.csv')
perfil_digital_202201_202204 = pd.read_csv(file_path+'03_perfil_digital_202201_202204.csv')#
adenda_202201_202204 = pd.read_csv(file_path+'02_adenda_202201_202204.csv')#
suscriptora_202201_202204 = pd.read_csv(file_path+'01_suscriptora_202201_202204.csv')#

# target

In [7]:
target=target_202201_202203.copy()
target['key_cliente']=target.PERIODO.astype(str)+'_'+target.nro_telefono_hash.astype(str)
target.drop(['PERIODO','nro_telefono_hash'],axis=1,inplace=True)
#target_202201_202203['TARGET']=target_202201_202203['TARGET'].astype('int')

# suscriptora

In [8]:
suscriptora=suscriptora_202201_202204.copy()
suscriptora['key_cliente']=suscriptora.NUMPERIODO.astype(str)+'_'+suscriptora.nro_telefono_hash.astype(str)
suscriptora['key_documento']=suscriptora.NUMPERIODO.astype(str)+'_'+suscriptora.nro_documento_hash.astype(str)

suscriptora=parse_month(suscriptora,['NUMPERIODO'],newcol=True)
suscriptora['NUMPERIODO_parsed'] =pd.to_datetime(suscriptora['NUMPERIODO_parsed'] , format='%Y/%m/%d')
suscriptora.shape

#suscriptora.FECINGRESOCLIENTE.str.slice(0,7).value_counts().sort_index()[0:50]
for mes in tqdm(suscriptora.NUMPERIODO_parsed.unique()):
    print(mes)
    suscriptora.loc[suscriptora['NUMPERIODO_parsed'] == mes,'NUMPERIODO_parsed']=pd.to_datetime(mes)+ relativedelta(months=1) - relativedelta(days=1)
    
suscriptora['FECINGRESOCLIENTE']=np.where(suscriptora['FECINGRESOCLIENTE'].isin(['0001-01-01 00:00:00'])
                                           ,np.nan,suscriptora['FECINGRESOCLIENTE'])
suscriptora['FECINGRESOCLIENTE'] = pd.to_datetime(suscriptora['FECINGRESOCLIENTE'] , format='%Y-%m-%d')



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

2022-01-01T00:00:00.000000000


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 14.10it/s]


2022-02-01T00:00:00.000000000
2022-03-01T00:00:00.000000000
2022-04-01T00:00:00.000000000


In [9]:
datetime_fold=suscriptora[suscriptora['FECINGRESOCLIENTE']>pd.to_datetime('2022-04-30 00:00:00')]
for mes in tqdm(datetime_fold['FECINGRESOCLIENTE'].unique()):
    #print(mes)
    suscriptora.loc[suscriptora['FECINGRESOCLIENTE'] == mes,'FECINGRESOCLIENTE']=np.nan

suscriptora['tiempo_entel']=(suscriptora.NUMPERIODO_parsed-suscriptora.FECINGRESOCLIENTE).astype('<m8[M]') 

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 78.21it/s]


In [10]:
suscriptora['aniomes_fic']=suscriptora.FECINGRESOCLIENTE.astype(str).str.slice(0,7)#.value_counts().sort_index()[240:]
suscriptora['aniomes_fac']=suscriptora.FECACTIVACIONCONTRATO.astype(str).str.slice(0,7)#.value_counts().sort_index()[240:]

In [11]:
# TRANSFORMANDO LAS VARIABLES DE FECHAS A MESES
#suscriptora.FECACTIVACIONCONTRATO.str.slice(0,7).value_counts().sort_index()[0:50]
suscriptora['FECACTIVACIONCONTRATO']=np.where(suscriptora['FECACTIVACIONCONTRATO'].isin(['0001-01-01 00:00:00'])
                                                                           ,np.nan,suscriptora['FECACTIVACIONCONTRATO'])
suscriptora['FECACTIVACIONCONTRATO'] = pd.to_datetime(suscriptora['FECACTIVACIONCONTRATO'] , format='%Y-%m-%d')

suscriptora['tiempo_contrato']=suscriptora.NUMPERIODO_parsed-suscriptora.FECACTIVACIONCONTRATO
suscriptora['tiempo_contrato']=suscriptora['tiempo_contrato'].astype('<m8[M]')
suscriptora['tiempo_contrato_ingreso']=suscriptora.FECACTIVACIONCONTRATO-suscriptora.FECINGRESOCLIENTE
suscriptora['tiempo_contrato_ingreso']=suscriptora['tiempo_contrato_ingreso'].astype('<m8[M]')


In [12]:
# nro de clientes por Tipo de adquisión
suscriptora['TIPO_ADQ_CALC']=suscriptora.TIPO_ADQ
for mes in tqdm(suscriptora['NUMPERIODO'].unique()):
    NCLIENTES_TIPO_ADQUI=suscriptora.loc[(suscriptora.NUMPERIODO==mes)].TIPO_ADQ.value_counts()
    suscriptora.loc[suscriptora.NUMPERIODO==mes,'TIPO_ADQ_CALC']=suscriptora.loc[suscriptora.NUMPERIODO==mes,'TIPO_ADQ'].map(NCLIENTES_TIPO_ADQUI)
suscriptora['TIPO_ADQ_CALC']=suscriptora['TIPO_ADQ_CALC'].astype('int')

# nro de clientes por Tipo de adquisión
suscriptora['aniomes_fic_cal']=suscriptora.aniomes_fic
for mes in tqdm(suscriptora['NUMPERIODO'].unique()):
    nro_cli_aniomes_fic=suscriptora.loc[(suscriptora.NUMPERIODO==mes)].aniomes_fic.value_counts()
    suscriptora.loc[suscriptora.NUMPERIODO==mes,'aniomes_fic_cal']=suscriptora.loc[suscriptora.NUMPERIODO==mes,'aniomes_fic'].map(nro_cli_aniomes_fic)
suscriptora['aniomes_fic_cal']=suscriptora['aniomes_fic_cal'].astype('int')

# nro de clientes por Tipo de adquisión
suscriptora['aniomes_fac_cal']=suscriptora.aniomes_fac
for mes in tqdm(suscriptora['NUMPERIODO'].unique()):
    nro_cli_aniomes_fac=suscriptora.loc[(suscriptora.NUMPERIODO==mes)].aniomes_fac.value_counts()
    suscriptora.loc[suscriptora.NUMPERIODO==mes,'aniomes_fac_cal']=suscriptora.loc[suscriptora.NUMPERIODO==mes,'aniomes_fac'].map(nro_cli_aniomes_fac)
suscriptora['aniomes_fac_cal']=suscriptora['aniomes_fac_cal'].astype('int')

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.70it/s]


In [13]:
def get_suscrip_features_single(doc,periodo):
    q1 = suscriptora.query('nro_documento_hash=="{}" and NUMPERIODO=={}'.format(doc,periodo))
    return q1

usg, usgf = [],[]
df1=suscriptora[['NUMPERIODO','nro_documento_hash']].drop_duplicates()
for doc,periodo in tqdm(zip(df1['nro_documento_hash'],df1['NUMPERIODO'])):
    #print(doc,periodo)
    a = get_suscrip_features_single(doc,periodo)
    b=a.groupby('TIPO_ADQ')['TIPO_ADQ'].transform('count').rename('tipo_adq_counts')
    usg.append(pd.concat([a, b], axis=1))
suscriptora=pd.concat(usg)

230236it [3:09:13, 20.28it/s]


# UNION DE LAS PRIMERAS TABLAS

In [14]:
# UNION DE SUSCRIPTORA Y TARGET
###########################################################
suscriptora.drop(['FECINGRESOCLIENTE','FECACTIVACIONCONTRATO'
                              ,'NUMPERIODO_parsed','aniomes_fic','aniomes_fac'],axis=1,inplace=True)
df_full_hist=pd.merge(suscriptora,target,on=['key_cliente'],how='left')

In [15]:
### CALCULO DEL DESERTOR POR TELEFONO EN LA HISTORIA ROLLING
desertor_hist= []  
hist=df_full_hist[['NUMPERIODO','nro_telefono_hash']]
hist_fugas=df_full_hist[(df_full_hist.TARGET==1)][['NUMPERIODO','nro_telefono_hash']] # fugas hist
for mes in tqdm(hist['NUMPERIODO'].unique()):
    partial=hist[hist['NUMPERIODO']==mes][['NUMPERIODO','nro_telefono_hash']]
    partial=partial.drop_duplicates()
    q = hist_fugas[(hist_fugas['nro_telefono_hash'].isin(partial['nro_telefono_hash'].unique())) 
            & (hist_fugas['NUMPERIODO']<mes)      
                    ]
    q1=q.groupby('nro_telefono_hash').count().add_suffix('_nro_salidas').fillna(0)
    desertor_hist.append(
     partial.merge(q1,on='nro_telefono_hash',how='left')
    )
desertor_hist = pd.concat(desertor_hist,sort=False).fillna(0)
desertor_hist.drop_duplicates(inplace=True)
desertor_hist['key_cliente']=desertor_hist.NUMPERIODO.astype(str)+'_'+desertor_hist.nro_telefono_hash.astype(str)
# renombramos 
desertor_hist.rename(columns={'NUMPERIODO_nro_salidas':'renovo_hist'},inplace=True)
###############################################3
# agregamos a la tabla general
desertor_hist.drop(['NUMPERIODO','nro_telefono_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,desertor_hist,on=['key_cliente'],how='left')
df_full_hist.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.90it/s]


(734544, 15)

In [16]:
suscriptora=pd.merge(suscriptora,df_full_hist[['key_cliente','renovo_hist']],on=['key_cliente'],how='left')

In [17]:
# CREANDO VARIABLES EN FUNCION AL TIP_ADQ
suscrip = []
for mes in tqdm(suscriptora['NUMPERIODO'].unique()):
        partial = suscriptora[suscriptora['NUMPERIODO']==mes][['NUMPERIODO','nro_documento_hash']] # verano de enero
        q = suscriptora[(suscriptora['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                       & (suscriptora.NUMPERIODO==mes)]
        
        q1 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['nro_documento_hash'].count().unstack().add_prefix('count_d_').reset_index().fillna(0)
        q2 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_entel'].mean().unstack().add_prefix('t_e_prom_d_').reset_index().fillna(0)
        q3 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_entel'].min().unstack().add_prefix('t_e_min_d_').reset_index().fillna(0)
        q4 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_entel'].max().unstack().add_prefix('t_e_max_d_').reset_index().fillna(0)
        q5 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_entel'].std().unstack().add_prefix('t_e_std_d_').reset_index().fillna(0)

        q6 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato'].mean().unstack().add_prefix('t_c_prom_d_').reset_index().fillna(0)
        q7 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato'].min().unstack().add_prefix('t_c_min_d_').reset_index().fillna(0)
        q8 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato'].max().unstack().add_prefix('t_c_max_d_').reset_index().fillna(0)
        q9 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato'].std().unstack().add_prefix('t_c_std_d_').reset_index().fillna(0)

        q10 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato_ingreso'].mean().unstack().add_prefix('t_c_i_prom_d_').reset_index().fillna(0)
        q11 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato_ingreso'].min().unstack().add_prefix('t_c_i_min_d_').reset_index().fillna(0)
        q12 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato_ingreso'].max().unstack().add_prefix('t_c_i_max_d_').reset_index().fillna(0)
        q13 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['tiempo_contrato_ingreso'].std().unstack().add_prefix('t_c_i_std_d_').reset_index().fillna(0)
        
        q14 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['renovo_hist'].mean().unstack().add_prefix('t_r_mean_d_').reset_index().fillna(0)
        q15 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['renovo_hist'].min().unstack().add_prefix('t_r_min_d_').reset_index().fillna(0)
        q16 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['renovo_hist'].max().unstack().add_prefix('t_r_max_d_').reset_index().fillna(0)
        q17 = q.groupby(['nro_documento_hash','TIPO_ADQ'])['renovo_hist'].std().unstack().add_prefix('t_r_std_d_').reset_index().fillna(0)



        suscrip.append(partial.merge(q1,on='nro_documento_hash',how='left').merge(q2,on='nro_documento_hash',how='left')
                           .merge(q3,on='nro_documento_hash',how='left').merge(q4,on='nro_documento_hash',how='left')
                       .merge(q5,on='nro_documento_hash',how='left').merge(q6,on='nro_documento_hash',how='left')
                       .merge(q7,on='nro_documento_hash',how='left').merge(q8,on='nro_documento_hash',how='left')
                       .merge(q9,on='nro_documento_hash',how='left').merge(q10,on='nro_documento_hash',how='left')
                       .merge(q11,on='nro_documento_hash',how='left').merge(q12,on='nro_documento_hash',how='left')
                       .merge(q13,on='nro_documento_hash',how='left').merge(q14,on='nro_documento_hash',how='left')
                       .merge(q15,on='nro_documento_hash',how='left').merge(q16,on='nro_documento_hash',how='left')
                       .merge(q17,on='nro_documento_hash',how='left')
                        )
suscrip = pd.concat(suscrip,sort=False)
suscrip=suscrip.drop_duplicates()
suscrip['key_documento']=suscrip.NUMPERIODO.astype(str)+'_'+suscrip.nro_documento_hash.astype(str)
#############################################################3
# UNION DE SUSCRIP
suscrip.drop(['NUMPERIODO','nro_documento_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,suscrip,on=['key_documento'],how='left')
df_full_hist.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:25<00:00,  6.35s/it]


(734544, 49)

In [18]:
# CREANDO VARIABLES 
suscrip2 = []
suscrip_tmp=suscriptora[['NUMPERIODO','nro_documento_hash','tiempo_entel','tiempo_contrato','tiempo_contrato_ingreso','renovo_hist']]
for mes in tqdm(suscrip_tmp['NUMPERIODO'].unique()):
        partial = suscrip_tmp[suscrip_tmp['NUMPERIODO']==mes][['NUMPERIODO','nro_documento_hash']] # verano de enero
        q = suscrip_tmp[(suscrip_tmp['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                       & (suscrip_tmp.NUMPERIODO==mes)]
        
        q1 = q.drop('NUMPERIODO',axis=1).groupby(['nro_documento_hash']).count().add_prefix('t_d_count_').reset_index().fillna(0)
        q2 = q.drop('NUMPERIODO',axis=1).groupby(['nro_documento_hash']).mean().add_prefix('t_d_prom_').reset_index().fillna(0)
        q3 = q.drop('NUMPERIODO',axis=1).groupby(['nro_documento_hash']).max().add_prefix('t_d_max_').reset_index().fillna(0)
        q4 = q.drop('NUMPERIODO',axis=1).groupby(['nro_documento_hash']).min().add_prefix('t_d_min_').reset_index().fillna(0)
        q5 = q.drop('NUMPERIODO',axis=1).groupby(['nro_documento_hash']).median().add_prefix('t_d_median_').reset_index().fillna(0)
        q6 = q.drop('NUMPERIODO',axis=1).groupby(['nro_documento_hash']).std().add_prefix('t_d_std_').reset_index().fillna(0)

        suscrip2.append(partial.merge(q1,on='nro_documento_hash',how='left').merge(q2,on='nro_documento_hash',how='left')
                        .merge(q3,on='nro_documento_hash',how='left').merge(q4,on='nro_documento_hash',how='left')
                        .merge(q5,on='nro_documento_hash',how='left').merge(q6,on='nro_documento_hash',how='left')
                        )
suscrip2 = pd.concat(suscrip2,sort=False)
suscrip2=suscrip2.drop_duplicates()
suscrip2['key_documento']=suscrip2.NUMPERIODO.astype(str)+'_'+suscrip2.nro_documento_hash.astype(str)
#############################################################3
# UNION DE SUSCRIP
suscrip2.drop(['NUMPERIODO','nro_documento_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,suscrip2,on=['key_documento'],how='left')
df_full_hist.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.97s/it]


(734544, 73)

In [19]:
# FUGAS POR TIPO DE ADQUIZ - y telefono
fugas = []
for mes in tqdm(df_full_hist['NUMPERIODO'].unique()):
        partial = df_full_hist[df_full_hist['NUMPERIODO']==mes][['key_cliente']] # verano de enero
        q = df_full_hist[(df_full_hist['key_cliente'].isin(partial['key_cliente'].unique()))]
        q1 = q.groupby(['key_cliente','TIPO_ADQ'])['renovo_hist'].sum().unstack().add_prefix('cont_fugas_').reset_index().fillna(0)
        fugas.append(partial.merge(q1,on='key_cliente',how='left')
                        )
fugas = pd.concat(fugas,sort=False)
fugas=fugas.drop_duplicates()
df_full_hist=pd.merge(df_full_hist,fugas,on=['key_cliente'],how='left')
df_full_hist.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.85s/it]


(734544, 75)

# tabla de digital

#### nro_telefono_hash: Código de identificación de la línea.
#### GRUPO: Categoria de aplicaciones usadas por la línea.
#### SCORECAT: Clasificación de que tan digital es el cliente categorizados por niveles.
#### PERIODO: Periodo de cierre de mes.

In [20]:
perfil_digital=perfil_digital_202201_202204.copy()
perfil_digital['key_cliente']=perfil_digital.PERIODO.astype(str)+'_'+perfil_digital.nro_telefono_hash.astype(str)
perfil_digital=pd.merge(perfil_digital,df_full_hist[['key_cliente','nro_documento_hash']],on=['key_cliente'],how='left')
perfil_digital['key_documento']=perfil_digital.PERIODO.astype(str)+'_'+perfil_digital.nro_documento_hash.astype(str)

dic = {'bajo': 1, 'medio': 2,'alto': 3, 'muy alto': 4}
perfil_digital['scortCat_val']= perfil_digital['SCORECAT']
perfil_digital['scortCat_val']=perfil_digital['SCORECAT'].map(dic)

In [ ]:
def get_digital_features_single(doc,periodo):
    q1 = perfil_digital.query('nro_documento_hash=="{}" and PERIODO=={}'.format(doc,periodo))
    return q1

usg, usgf = [],[]
df1=perfil_digital[['PERIODO','nro_documento_hash']].drop_duplicates()
for doc,periodo in tqdm(zip(df1['nro_documento_hash'],df1['PERIODO'])):
    #print(doc,periodo)
    a = get_digital_features_single(doc,periodo)
    b=a.groupby('SCORECAT')['SCORECAT'].transform('count').rename('scorecat_counts')
    usg.append(pd.concat([a, b], axis=1))
perfil_digital=pd.concat(usg)

113808it [2:03:55, 16.21it/s]

In [ ]:
# nro de clientes por Tipo de categoria
perfil_digital['SCORECAT_CALC']=perfil_digital.SCORECAT
for mes in tqdm(perfil_digital['PERIODO'].unique()):
    NCLIENTES_TIPO_CATE=perfil_digital.loc[(perfil_digital.PERIODO==mes)].SCORECAT.value_counts()
    perfil_digital.loc[perfil_digital.PERIODO==mes,'SCORECAT_CALC']=perfil_digital.loc[perfil_digital.PERIODO==mes,'SCORECAT'].map(NCLIENTES_TIPO_CATE)
perfil_digital['SCORECAT_CALC']=perfil_digital['SCORECAT_CALC'].astype('int')

In [ ]:
# TRANSFORMAION DE LA VARIABLE CADENA DE DATOS A COLUMNAS
perfil_digital=pd.concat([perfil_digital,perfil_digital.GRUPO.str.get_dummies(sep = "|")], axis =1)

In [ ]:
# SUMANDO TODAS LAS CATEGORIAS USADAS
grupo_total=('grupo_1','grupo_2','grupo_3','grupo_4','grupo_5','grupo_6','grupo_7','grupo_8','grupo_9','grupo_10','grupo_11')
perfil_digital['grupo_total']=perfil_digital.loc[:,grupo_total].sum(axis=1) # 
#perfil_digital.drop(['grupo_1','grupo_2','grupo_3','grupo_4','grupo_5','grupo_6','grupo_7','grupo_8','grupo_9','grupo_10','grupo_11'],axis=1,inplace=True)


# por telefono

In [ ]:
perfil_digital=pd.merge(perfil_digital,df_full_hist[['key_cliente','renovo_hist']],on=['key_cliente'],how='left')

In [ ]:
# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo telefono
digital = []
for mes in tqdm(perfil_digital['PERIODO'].unique()):
        partial = perfil_digital[perfil_digital['PERIODO']==mes][['key_cliente']] # verano de enero
        q = perfil_digital[(perfil_digital['key_cliente'].isin(partial['key_cliente'].unique()))]
        q1 = q.groupby(['key_cliente','SCORECAT'])['grupo_total'].sum().unstack().add_prefix('sum_digi_').reset_index().fillna(0)
        q2 = q.groupby(['key_cliente','SCORECAT'])['renovo_hist'].sum().unstack().add_prefix('sum_rc_').reset_index().fillna(0)

        digital.append(partial.merge(q1,on='key_cliente',how='left').merge(q1,on='key_cliente',how='left')
                        )
digital = pd.concat(digital,sort=False)
digital=digital.drop_duplicates()

In [ ]:
# SCORT DE CATEGORIAS USADAS ROLLING POR TIPO DE USUARIO TELEFONO
# el max , min y prom de clasificacion que obtuvo en su historia
scortCat_hist= []  
hist=perfil_digital[['PERIODO','nro_telefono_hash','scortCat_val','grupo_total','renovo_hist']]
for mes in tqdm(hist['PERIODO'].unique()):
    partial=hist[hist['PERIODO']==mes][['PERIODO','nro_telefono_hash']]
    partial=partial.drop_duplicates()
    q = hist[(hist['nro_telefono_hash'].isin(partial['nro_telefono_hash'].unique())) 
            & (hist['PERIODO']<=mes)
                    ]
    
    q1=q.drop('PERIODO',axis=1).groupby('nro_telefono_hash').max().add_suffix('_nro_max').fillna(0)
    q2=q.drop('PERIODO',axis=1).groupby('nro_telefono_hash').min().add_suffix('_nro_min').fillna(0)
    q3=q.drop('PERIODO',axis=1).groupby('nro_telefono_hash').mean().add_suffix('_nro_prom').fillna(0)
    q4=q.drop('PERIODO',axis=1).groupby('nro_telefono_hash').sum().add_suffix('_nro_sum').fillna(0)

    
    scortCat_hist.append(
     partial.merge(q1,on='nro_telefono_hash',how='left').merge(q2,on='nro_telefono_hash',how='left')
        .merge(q3,on='nro_telefono_hash',how='left').merge(q4,on='nro_telefono_hash',how='left')
    )
scortCat_hist = pd.concat(scortCat_hist,sort=False).fillna(0)
scortCat_hist.drop_duplicates(inplace=True)
scortCat_hist['key_cliente']=scortCat_hist.PERIODO.astype(str)+'_'+scortCat_hist.nro_telefono_hash.astype(str)
#desertor_hist.drop(['NUMPERIODO','nro_telefono_hash'],axis=1,inplace=True)

In [ ]:
# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo persona
digital_cli = []
for mes in tqdm(perfil_digital['PERIODO'].unique()):
        partial = perfil_digital[perfil_digital['PERIODO']==mes][['PERIODO','nro_telefono_hash']] # verano de enero
        q = perfil_digital[(perfil_digital['nro_telefono_hash'].isin(partial['nro_telefono_hash'].unique()))
                           & (perfil_digital.PERIODO<=mes)]
        
        q1 = q.groupby(['nro_telefono_hash','SCORECAT'])['grupo_total'].sum().unstack().add_prefix('sum_digi_cli_').reset_index().fillna(0)
        q2 = q.groupby(['nro_telefono_hash','SCORECAT'])['grupo_total'].mean().unstack().add_prefix('mean_digi_cli_').reset_index().fillna(0)
        q3 = q.groupby(['nro_telefono_hash','SCORECAT'])['grupo_total'].min().unstack().add_prefix('min_digi_cli_').reset_index().fillna(0)
        q4 = q.groupby(['nro_telefono_hash','SCORECAT'])['grupo_total'].max().unstack().add_prefix('max_digi_cli_').reset_index().fillna(0)
        q5 = q.groupby(['nro_telefono_hash','SCORECAT'])['grupo_total'].std().unstack().add_prefix('std_digi_cli_').reset_index().fillna(0)
        
        q6 = q.groupby(['nro_telefono_hash','SCORECAT'])['renovo_hist'].sum().unstack().add_prefix('sum_digi_rc_').reset_index().fillna(0)
        q7 = q.groupby(['nro_telefono_hash','SCORECAT'])['renovo_hist'].mean().unstack().add_prefix('mean_digi_rc_').reset_index().fillna(0)
        q8 = q.groupby(['nro_telefono_hash','SCORECAT'])['renovo_hist'].min().unstack().add_prefix('min_digi_rc_').reset_index().fillna(0)
        q9 = q.groupby(['nro_telefono_hash','SCORECAT'])['renovo_hist'].max().unstack().add_prefix('max_digi_rc_').reset_index().fillna(0)
        q10 = q.groupby(['nro_telefono_hash','SCORECAT'])['renovo_hist'].std().unstack().add_prefix('std_digi_rc_').reset_index().fillna(0)
        digital_cli.append(partial.merge(q1,on='nro_telefono_hash',how='left')
                           .merge(q2,on='nro_telefono_hash',how='left').merge(q3,on='nro_telefono_hash',how='left')
                           .merge(q4,on='nro_telefono_hash',how='left').merge(q5,on='nro_telefono_hash',how='left')
                           .merge(q6,on='nro_telefono_hash',how='left').merge(q7,on='nro_telefono_hash',how='left')
                           .merge(q8,on='nro_telefono_hash',how='left').merge(q9,on='nro_telefono_hash',how='left')
                           .merge(q10,on='nro_telefono_hash',how='left')
                        )
digital_cli = pd.concat(digital_cli,sort=False)
digital_cli=digital_cli.drop_duplicates()
digital_cli['key_cliente']=digital_cli.PERIODO.astype(str)+'_'+digital_cli.nro_telefono_hash.astype(str)

In [ ]:
# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo persona
digital_doc = []
for mes in tqdm(perfil_digital['PERIODO'].unique()):
        partial = perfil_digital[perfil_digital['PERIODO']==mes][['PERIODO','nro_documento_hash']] # verano de enero
        q = perfil_digital[(perfil_digital['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                           & (perfil_digital.PERIODO==mes)]
        q_ = perfil_digital[(perfil_digital['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                           & (perfil_digital.PERIODO<=mes)]
        
        q1 = q.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].sum().unstack().add_prefix('sum_digi_doc_').reset_index().fillna(0)
        q2 = q.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].mean().unstack().add_prefix('mean_digi_doc_').reset_index().fillna(0)
        q3 = q_.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].sum().unstack().add_prefix('sum_digi_doc2_').reset_index().fillna(0)
        q4 = q_.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].mean().unstack().add_prefix('mean_digi_doc2_').reset_index().fillna(0)
        q5 = q.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].min().unstack().add_prefix('min_digi_doc_').reset_index().fillna(0)
        q6 = q_.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].min().unstack().add_prefix('min_digi_doc2_').reset_index().fillna(0)
        q7 = q.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].max().unstack().add_prefix('max_digi_doc_').reset_index().fillna(0)
        q8 = q_.groupby(['nro_documento_hash','SCORECAT'])['grupo_total'].max().unstack().add_prefix('max_digi_doc2_').reset_index().fillna(0)
        
        q9 = q.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].sum().unstack().add_prefix('sum_digi_rc_').reset_index().fillna(0)
        q10 = q.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].mean().unstack().add_prefix('mean_digi_rc_').reset_index().fillna(0)
        q11 = q_.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].sum().unstack().add_prefix('sum_digi_rc_').reset_index().fillna(0)
        q12 = q_.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].mean().unstack().add_prefix('mean_digi_rc_').reset_index().fillna(0)
        q13 = q.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].min().unstack().add_prefix('min_digi_rc_').reset_index().fillna(0)
        q14 = q_.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].min().unstack().add_prefix('min_digi_rc_').reset_index().fillna(0)
        q15 = q.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].max().unstack().add_prefix('max_digi_rc_').reset_index().fillna(0)
        q16 = q_.groupby(['nro_documento_hash','SCORECAT'])['renovo_hist'].max().unstack().add_prefix('max_digi_rc_').reset_index().fillna(0)

        digital_doc.append(partial.merge(q1,on='nro_documento_hash',how='left')
                           .merge(q2,on='nro_documento_hash',how='left').merge(q3,on='nro_documento_hash',how='left')
                           .merge(q4,on='nro_documento_hash',how='left').merge(q5,on='nro_documento_hash',how='left')
                           .merge(q6,on='nro_documento_hash',how='left').merge(q7,on='nro_documento_hash',how='left')
                           .merge(q8,on='nro_documento_hash',how='left').merge(q9,on='nro_documento_hash',how='left')
                           .merge(q10,on='nro_documento_hash',how='left').merge(q11,on='nro_documento_hash',how='left')
                           .merge(q12,on='nro_documento_hash',how='left').merge(q13,on='nro_documento_hash',how='left')
                           .merge(q14,on='nro_documento_hash',how='left').merge(q15,on='nro_documento_hash',how='left')
                           .merge(q16,on='nro_documento_hash',how='left')
                        )
digital_doc = pd.concat(digital_doc,sort=False)
digital_doc=digital_doc.drop_duplicates()
digital_doc['key_documento']=digital_doc.PERIODO.astype(str)+'_'+digital_doc.nro_documento_hash.astype(str)

In [ ]:

scortCat_doc_hist= []  
hist=perfil_digital[['PERIODO','nro_documento_hash','scortCat_val','grupo_total','renovo_hist']]
for mes in tqdm(hist['PERIODO'].unique()):
    partial=hist[hist['PERIODO']==mes][['PERIODO','nro_documento_hash']]
    partial=partial.drop_duplicates()
    q = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['PERIODO']<=mes)  
            ]
    q_ = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['PERIODO']==mes) 
                    ]
    q1=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').max().add_suffix('_doc_snro_max').fillna(0)
    q2=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').min().add_suffix('_doc_snro_min').fillna(0)
    q3=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').mean().add_suffix('_doc_snro_prom').fillna(0)
    q4=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').sum().add_suffix('_doc_snro_sum').fillna(0)
    q5=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').std().add_suffix('_doc_snro_std').fillna(0)

       
    q6=q_.drop('PERIODO',axis=1).groupby('nro_documento_hash').max().add_suffix('_doc_snro_max_2').fillna(0)
    q7=q_.drop('PERIODO',axis=1).groupby('nro_documento_hash').min().add_suffix('_doc_snro_min_2').fillna(0)
    q8=q_.drop('PERIODO',axis=1).groupby('nro_documento_hash').mean().add_suffix('_doc_snro_prom_2').fillna(0)
    q9=q_.drop('PERIODO',axis=1).groupby('nro_documento_hash').sum().add_suffix('_doc_snro_sum_2').fillna(0)
    q10=q_.drop('PERIODO',axis=1).groupby('nro_documento_hash').std().add_suffix('_doc_snro_std_2').fillna(0)


    
    scortCat_doc_hist.append(
     partial.merge(q1,on='nro_documento_hash',how='left').merge(q2,on='nro_documento_hash',how='left')
        .merge(q3,on='nro_documento_hash',how='left').merge(q4,on='nro_documento_hash',how='left')
        .merge(q5,on='nro_documento_hash',how='left').merge(q6,on='nro_documento_hash',how='left')
        .merge(q7,on='nro_documento_hash',how='left').merge(q8,on='nro_documento_hash',how='left')
        .merge(q9,on='nro_documento_hash',how='left').merge(q10,on='nro_documento_hash',how='left')
    )
scortCat_doc_hist = pd.concat(scortCat_doc_hist,sort=False).fillna(0)
scortCat_doc_hist.drop_duplicates(inplace=True)
scortCat_doc_hist['key_documento']=scortCat_doc_hist.PERIODO.astype(str)+'_'+scortCat_doc_hist.nro_documento_hash.astype(str)
#desertor_hist.drop(['NUMPERIODO','nro_telefono_hash'],axis=1,inplace=True)

In [ ]:
# UNION DE LAS TABLAS CREADAS DE DIGITAL
perfil_digital.drop(['PERIODO','nro_telefono_hash','nro_documento_hash','key_documento','GRUPO','SCORECAT'],axis=1,inplace=True)
digital_doc.drop(['PERIODO','nro_documento_hash'],axis=1,inplace=True)
digital_cli.drop(['PERIODO','nro_telefono_hash'],axis=1,inplace=True)
scortCat_hist.drop(['PERIODO','nro_telefono_hash'],axis=1,inplace=True)
scortCat_doc_hist.drop(['PERIODO','nro_documento_hash'],axis=1,inplace=True)


df_full_hist=pd.merge(df_full_hist,perfil_digital,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,digital,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,digital_doc,on=['key_documento'],how='left')
df_full_hist=pd.merge(df_full_hist,digital_cli,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,scortCat_hist,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,scortCat_doc_hist,on=['key_documento'],how='left')

df_full_hist.shape

# ADENDAS

##### VCHMESADENDA: Meses transcurridos desde el inicio de la adenda.
##### VCHPENALIDAD: Monto de penalidad vigente a la fecha en caso de desactivación.

In [ ]:
adenda=adenda_202201_202204.copy()
adenda['key_cliente']=adenda.NUMPERIODO.astype(str)+'_'+adenda.nro_telefono_hash.astype(str)
adenda=pd.merge(adenda,df_full_hist[['key_cliente','nro_documento_hash']],on=['key_cliente'],how='left')
adenda['key_documento']=adenda.NUMPERIODO.astype(str)+'_'+adenda.nro_documento_hash.astype(str)

In [ ]:
def get_adenda_features_single(key_doc,mes_adenda,penalidad,adenda):
    q1 = adenda.query('key_documento=="{}" '.format(key_doc))
    return (mes_adenda < q1['VCHMESADENDA']).mean(), (penalidad < q1['VCHPENALIDAD']).mean()

def get_adenda_features(df,adenda):
    # No es la forma mas rápida, pero hace el trabajo
    usg, usgf = [],[]
    adenda['VCHMESADENDA'] = adenda['VCHMESADENDA'].fillna(-999)
    adenda['VCHPENALIDAD'] = adenda['VCHPENALIDAD'].fillna(-999)

    for key_doc,mes_adenda,penalidad in tqdm(zip(adenda['key_documento'],adenda['VCHMESADENDA'],adenda['VCHPENALIDAD'])):
        #print(key_doc,mes_adenda,penalidad)
        a,b = get_adenda_features_single(key_doc,mes_adenda,penalidad,adenda)
        usg.append(a)
        usgf.append(b)
    adenda['rel_use'] = usg
    adenda['rel_use_full'] = usgf
    
    adenda['key_documento_count'] = np.nan
    for m in tqdm(adenda['key_documento'].unique()):
        adenda['key_documento_count'] = np.where(adenda['key_documento']==m,sum(adenda['key_documento']==m),adenda['key_documento_count'])
        
    adenda['VCHMESADENDA'] = adenda['VCHMESADENDA'].replace(-999,np.nan)
    adenda['VCHPENALIDAD'] = adenda['VCHPENALIDAD'].replace(-999,np.nan)
  
    agg = {'key_documento' : ['count'],
                   'rel_use' : ['mean'],
           'rel_use_full' : ['mean']}
    
    adendap = adenda.groupby('key_documento').agg(agg).reset_index()
    adendap.columns = ['key_documento']+[a+'_'+b for a,b in adendap.columns[1:]]

    return df.merge(adendap,on='key_documento',how='left')
df_full_hist = get_adenda_features(df_full_hist, adenda)

adenda1=adenda[adenda.key_documento.isin(['202201_7759b41a7d1df4b2d8f721577abc24a48d95dc5f01fbab5937e9b3f6bfe84fc6'
                                  ,'202202_f3d5b2a430a3fd220d08e43875ab977aaacbf4e7b167c9993804e5d5e15caafc'
                                         ,'202201_b8627a8bd1b8e4e6f05fa2adb87e51cea0ffe232ae0b117ea6604b89f123fa4a'
                                         ,'202201_bcddf613461efae35b7001611e394258b231b88cbdb61c03123634ce48682528'])]
get_adenda_features(df_full_hist,adenda1)

In [ ]:
# estadisticas a nivel de documento

adenda_doc_hist= []  
hist=adenda[['NUMPERIODO','nro_documento_hash','VCHMESADENDA','VCHPENALIDAD']]
for mes in tqdm(hist['NUMPERIODO'].unique()):
    partial=hist[hist['NUMPERIODO']==mes][['NUMPERIODO','nro_documento_hash']]
    partial=partial.drop_duplicates()
    q = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['NUMPERIODO']<=mes)      
                    ]
    q_ = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['NUMPERIODO']==mes)
             ]
    q1=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').max().add_suffix('_doc_nro_max').fillna(0)
    q2=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').min().add_suffix('_doc_nro_min').fillna(0)
    q3=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').mean().add_suffix('_doc_nro_prom').fillna(0)
    q4=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').sum().add_suffix('_doc_nro_sum').fillna(0)
    q5=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').std().add_suffix('_doc_nro_std').fillna(0)


    
    q6=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').max().add_suffix('_doc_nro_max2').fillna(0)
    q7=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').min().add_suffix('_doc_nro_min2').fillna(0)
    q8=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').mean().add_suffix('_doc_nro_prom2').fillna(0)
    q9=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').sum().add_suffix('_doc_nro_sum2').fillna(0)
    q10=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').std().add_suffix('_doc_nro_std2').fillna(0)



    adenda_doc_hist.append(
     partial.merge(q1,on='nro_documento_hash',how='left').merge(q2,on='nro_documento_hash',how='left')
        .merge(q3,on='nro_documento_hash',how='left').merge(q4,on='nro_documento_hash',how='left')
        .merge(q5,on='nro_documento_hash',how='left').merge(q6,on='nro_documento_hash',how='left')
        .merge(q7,on='nro_documento_hash',how='left').merge(q8,on='nro_documento_hash',how='left')
        .merge(q9,on='nro_documento_hash',how='left').merge(q10,on='nro_documento_hash',how='left')
    )
adenda_doc_hist = pd.concat(adenda_doc_hist,sort=False).fillna(0)
adenda_doc_hist.drop_duplicates(inplace=True)
adenda_doc_hist['key_documento']=adenda_doc_hist.NUMPERIODO.astype(str)+'_'+adenda_doc_hist.nro_documento_hash.astype(str)

In [ ]:
# UNION DE LAS TABLAS CREADAS DE adendas
adenda.drop(['NUMPERIODO','nro_telefono_hash','nro_documento_hash','key_documento'],axis=1,inplace=True)
#adenda_hist.drop(['NUMPERIODO','nro_telefono_hash'],axis=1,inplace=True)
adenda_doc_hist.drop(['NUMPERIODO','nro_documento_hash'],axis=1,inplace=True)

#df_full_hist.drop(['NUMPERIODO','nro_telefono_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,adenda,on=['key_cliente'],how='left')
#df_full_hist=pd.merge(df_full_hist,adenda_hist,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,adenda_doc_hist,on=['key_documento'],how='left')

In [ ]:
print(df_full_hist.shape)

# TRAFICO

In [ ]:
trafico=trafico_202201_202204.copy()
trafico['key_cliente']=trafico.NUMPERIODO.astype(str)+'_'+trafico.nro_telefono_hash.astype(str)
trafico=pd.merge(trafico,df_full_hist[['key_cliente','nro_documento_hash']],on=['key_cliente'],how='left')

In [ ]:
# SUMANDO TODAS los traficos 
trafico_subtotal=('trafico_app_1','trafico_app_2' ,'trafico_app_7','trafico_app_8'
             ,'trafico_app_3','trafico_app_4','trafico_app_5','trafico_app_6','trafico_app_9')
trafico['trafico_subtotal']=trafico.loc[:,trafico_subtotal].sum(axis=1) 
trafico['peso_traf']=trafico['trafico_subtotal']/trafico['trafico_total']

mins_flujo=('mins_flujo_1','mins_flujo_2')
trafico['mins_flujo_total']=trafico.loc[:,mins_flujo].sum(axis=1) 
trafico['peso_flujo1']=trafico['mins_flujo_1']/trafico['mins_flujo_total']
trafico['peso_flujo2']=trafico['mins_flujo_2']/trafico['mins_flujo_total']

#trafico.drop(['trafico_app_1','trafico_app_2' ,'trafico_app_7','trafico_app_8'
             #,'trafico_app_3','trafico_app_4','trafico_app_5','trafico_app_6','trafico_app_9'
            # ,'mins_flujo_1','mins_flujo_2'],axis=1,inplace=True)


In [ ]:

traf_doc_hist= []  
hist=trafico[['NUMPERIODO','nro_documento_hash','trafico_subtotal','mins_flujo_total','trafico_total']]
for mes in tqdm(hist['NUMPERIODO'].unique()):
    partial=hist[hist['NUMPERIODO']==mes][['NUMPERIODO','nro_documento_hash']]
    partial=partial.drop_duplicates()
    q = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['NUMPERIODO']<=mes)      
                    ]
    q_ = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['NUMPERIODO']==mes)      
                    ]
    q1=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').mean().add_suffix('_doc_tr_prom').fillna(0)
    q2=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').sum().add_suffix('_doc_tr_sum').fillna(0)
    q3=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').min().add_suffix('_doc_tr_min').fillna(0)
    q4=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').max().add_suffix('_doc_tr_max').fillna(0)
    q5=q.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').std().add_suffix('_doc_tr_std').fillna(0)

    q6=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').mean().add_suffix('_doc_tr_prom2').fillna(0)
    q7=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').sum().add_suffix('_doc_tr_sum2').fillna(0)
    q8=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').min().add_suffix('_doc_tr_min2').fillna(0)
    q9=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').max().add_suffix('_doc_tr_max2').fillna(0)
    q10=q_.drop('NUMPERIODO',axis=1).groupby('nro_documento_hash').std().add_suffix('_doc_tr_std2').fillna(0)


    traf_doc_hist.append(
     partial.merge(q1,on='nro_documento_hash',how='left').merge(q2,on='nro_documento_hash',how='left')
        .merge(q3,on='nro_documento_hash',how='left').merge(q4,on='nro_documento_hash',how='left')
        .merge(q5,on='nro_documento_hash',how='left').merge(q6,on='nro_documento_hash',how='left')
        .merge(q7,on='nro_documento_hash',how='left').merge(q8,on='nro_documento_hash',how='left')
        .merge(q9,on='nro_documento_hash',how='left').merge(q10,on='nro_documento_hash',how='left')
    )
traf_doc_hist = pd.concat(traf_doc_hist,sort=False).fillna(0)
traf_doc_hist.drop_duplicates(inplace=True)
traf_doc_hist['key_documento']=traf_doc_hist.NUMPERIODO.astype(str)+'_'+traf_doc_hist.nro_documento_hash.astype(str)

In [ ]:

traf_tel_hist= []  
hist=trafico[['NUMPERIODO','nro_telefono_hash','trafico_subtotal','mins_flujo_total','trafico_total']]
for mes in tqdm(hist['NUMPERIODO'].unique()):
    partial=hist[hist['NUMPERIODO']==mes][['NUMPERIODO','nro_telefono_hash']]
    partial=partial.drop_duplicates()
    q = hist[(hist['nro_telefono_hash'].isin(partial['nro_telefono_hash'].unique())) 
            & (hist['NUMPERIODO']<=mes)      
                    ]
    q1=q.drop('NUMPERIODO',axis=1).groupby('nro_telefono_hash').mean().add_suffix('_tel_tr_prom').fillna(0)
    q2=q.drop('NUMPERIODO',axis=1).groupby('nro_telefono_hash').sum().add_suffix('_tel_tr_sum').fillna(0)
    q3=q.drop('NUMPERIODO',axis=1).groupby('nro_telefono_hash').min().add_suffix('_tel_tr_min').fillna(0)
    q4=q.drop('NUMPERIODO',axis=1).groupby('nro_telefono_hash').max().add_suffix('_tel_tr_max').fillna(0)
    q5=q.drop('NUMPERIODO',axis=1).groupby('nro_telefono_hash').std().add_suffix('_tel_tr_std').fillna(0)


    traf_tel_hist.append(
     partial.merge(q1,on='nro_telefono_hash',how='left').merge(q2,on='nro_telefono_hash',how='left')
        .merge(q3,on='nro_telefono_hash',how='left').merge(q4,on='nro_telefono_hash',how='left')
        .merge(q5,on='nro_telefono_hash',how='left')
    )
traf_tel_hist = pd.concat(traf_tel_hist,sort=False).fillna(0)
traf_tel_hist.drop_duplicates(inplace=True)
traf_tel_hist['key_cliente']=traf_tel_hist.NUMPERIODO.astype(str)+'_'+traf_tel_hist.nro_telefono_hash.astype(str)

In [ ]:
traf_tel_hist.drop(['NUMPERIODO','nro_telefono_hash'],axis=1,inplace=True)
traf_doc_hist.drop(['NUMPERIODO','nro_documento_hash'],axis=1,inplace=True)

df_full_hist=pd.merge(df_full_hist,traf_tel_hist,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,traf_doc_hist,on=['key_documento'],how='left')

In [ ]:
trafico.drop(['NUMPERIODO','nro_telefono_hash','nro_documento_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,trafico,on=['key_cliente'],how='left')
print(df_full_hist.shape)

# terminales

In [ ]:
def replace_low_classes(lst,thresh,string):
    c2r = lst.value_counts()[lst.value_counts()<thresh].index
    return lst.replace({x:string for x in c2r})

In [ ]:
terminales=terminales_202201_202204.copy()
terminales['key_cliente']=terminales.PERIODO.astype(str)+'_'+terminales.nro_telefono_hash.astype(str)
terminales=parse_month(terminales,['PERIODO'],newcol=True)
#terminales.drop(['PERIODO','nro_telefono_hash'],axis=1,inplace=True)


In [ ]:
terminales['LANZAMIENTO'] = pd.to_datetime(terminales['LANZAMIENTO'] , format='%Y-%m-%d')
datetime_fold=terminales[terminales['LANZAMIENTO']>pd.to_datetime('2022-04-30 00:00:00')]
for mes in tqdm(datetime_fold['LANZAMIENTO'].unique()):
    #print(mes)
    terminales.loc[terminales['LANZAMIENTO'] == mes,'LANZAMIENTO']=np.nan

In [ ]:
terminales['aniomes_lanz']=terminales.LANZAMIENTO.astype(str).str.slice(0,4)
# nro de clientes por Tipo de adquisión
terminales['aniomes_lanz_cal']=terminales.aniomes_lanz
for mes in tqdm(terminales['PERIODO'].unique()):
    nro_cli_aniomes_lanz=terminales.loc[(terminales.PERIODO==mes)].aniomes_lanz.value_counts()
    terminales.loc[terminales.PERIODO==mes,'aniomes_lanz_cal']=terminales.loc[terminales.PERIODO==mes,'aniomes_lanz'].map(nro_cli_aniomes_lanz)
terminales['aniomes_lanz_cal']=terminales['aniomes_lanz_cal'].astype('int')


In [ ]:
# transformacion para optener la variable de tiempo de lanzamiento
terminales['lanzamiento_equipo']=(terminales.PERIODO_parsed-terminales.LANZAMIENTO).astype('<m8[M]') 

In [ ]:
# Transfomracion de las variables categoricas
terminales['MARCA'] = replace_low_classes(terminales['MARCA'],thresh=500,string='Otros').values # 7 =7 a más
terminales['MODELO'] = replace_low_classes(terminales['MODELO'],thresh=500,string='Otros').values # 7 =7 a más
terminales['NUEVA_GAMMA'] = replace_low_classes(terminales['NUEVA_GAMMA'],thresh=500,string='Otros').values # 7 =7 a más
terminales['OS'] = replace_low_classes(terminales['OS'],thresh=500,string='Otros').values # 7 =7 a más
terminales['DEVICE_TYPE'] = replace_low_classes(terminales['DEVICE_TYPE'],thresh=500,string='Otros').values # 7 =7 a más

terminales['MARCA']=terminales['MARCA'].astype('category')
terminales['MODELO']=terminales['MODELO'].astype('category')
terminales['NUEVA_GAMMA']=terminales['NUEVA_GAMMA'].astype('category')
terminales['OS']=terminales['OS'].astype('category')
terminales['DEVICE_TYPE']=terminales['DEVICE_TYPE'].astype('category')

terminales['MARCA']= terminales['MARCA'].cat.codes
terminales['MODELO']= terminales['MODELO'].cat.codes
terminales['NUEVA_GAMMA']= terminales['NUEVA_GAMMA'].cat.codes
terminales['OS']= terminales['OS'].cat.codes
terminales['DEVICE_TYPE']= terminales['DEVICE_TYPE'].cat.codes

In [ ]:
# nro de clientes por Tipo de scort de categoria
terminales['MARCA_CALC']=terminales.MARCA
for mes in tqdm(terminales['PERIODO'].unique()):
    NCLIENTES_MARCA=terminales.loc[(terminales.PERIODO==mes)].MARCA.value_counts()
    terminales.loc[terminales.PERIODO==mes,'MARCA_CALC']=terminales.loc[terminales.PERIODO==mes,'MARCA'].map(NCLIENTES_MARCA)
terminales['MARCA_CALC']=terminales['MARCA_CALC'].astype('int')

terminales['MODELO_CALC']=terminales.MODELO
for mes in tqdm(terminales['PERIODO'].unique()):
    NCLIENTES_MODELO=terminales.loc[(terminales.PERIODO==mes)].MODELO.value_counts()
    terminales.loc[terminales.PERIODO==mes,'MODELO_CALC']=terminales.loc[terminales.PERIODO==mes,'MODELO'].map(NCLIENTES_MODELO)
terminales['MODELO_CALC']=terminales['MODELO_CALC'].astype('int')

terminales['NUEVA_GAMMA_CALC']=terminales.NUEVA_GAMMA
for mes in tqdm(terminales['PERIODO'].unique()):
    NCLIENTES_NUEVA_GAMMA=terminales.loc[(terminales.PERIODO==mes)].NUEVA_GAMMA.value_counts()
    terminales.loc[terminales.PERIODO==mes,'NUEVA_GAMMA_CALC']=terminales.loc[terminales.PERIODO==mes,'NUEVA_GAMMA'].map(NCLIENTES_NUEVA_GAMMA)
terminales['NUEVA_GAMMA_CALC']=terminales['NUEVA_GAMMA_CALC'].astype('int')

terminales['OS_CALC']=terminales.OS
for mes in tqdm(terminales['PERIODO'].unique()):
    NCLIENTES_OS=terminales.loc[(terminales.PERIODO==mes)].OS.value_counts()
    terminales.loc[terminales.PERIODO==mes,'OS_CALC']=terminales.loc[terminales.PERIODO==mes,'OS'].map(NCLIENTES_OS)
terminales['OS_CALC']=terminales['OS_CALC'].astype('int')

terminales['DEVICE_TYPE_CALC']=terminales.DEVICE_TYPE
for mes in tqdm(terminales['PERIODO'].unique()):
    NCLIENTES_DEVICE_TYPE=terminales.loc[(terminales.PERIODO==mes)].DEVICE_TYPE.value_counts()
    terminales.loc[terminales.PERIODO==mes,'DEVICE_TYPE_CALC']=terminales.loc[terminales.PERIODO==mes,'DEVICE_TYPE'].map(NCLIENTES_DEVICE_TYPE)
terminales['DEVICE_TYPE_CALC']=terminales['DEVICE_TYPE_CALC'].astype('int')

In [ ]:
terminales=pd.merge(terminales,df_full_hist[['key_cliente','nro_documento_hash']],on=['key_cliente'],how='left')

In [ ]:
def get_term_features_single(doc,periodo):
    q1 = terminales.query('nro_documento_hash=="{}" and PERIODO=={}'.format(doc,periodo))
    return q1

usg, usgf = [],[]
df1=terminales[['PERIODO','nro_documento_hash']].drop_duplicates()
for doc,periodo in tqdm(zip(df1['nro_documento_hash'],df1['PERIODO'])):
    #print(doc,periodo)
    a = get_term_features_single(doc,periodo)
    b=a.groupby('MARCA')['MARCA'].transform('count').rename('marca_counts')
    c=a.groupby('MODELO')['MODELO'].transform('count').rename('modelo_counts')
    d=a.groupby('NUEVA_GAMMA')['NUEVA_GAMMA'].transform('count').rename('ngama_counts')
    e=a.groupby('OS')['OS'].transform('count').rename('os_counts')
    f=a.groupby('DEVICE_TYPE')['DEVICE_TYPE'].transform('count').rename('device_counts')
    usg.append(pd.concat([a, b, c, d, e, f], axis=1))
terminales=pd.concat(usg)

def get_term_features_single(doc,periodo):
    q1 = terminales.query('nro_documento_hash=="{}" and PERIODO=={}'.format(doc,periodo))
    return q1

usg, usgf = [],[]
terminales1=terminales[['PERIODO','nro_documento_hash']].drop_duplicates()
for doc,periodo in tqdm(zip(terminales1['nro_documento_hash'],terminales1['PERIODO'])):
    #print(doc,periodo)
    a = get_term_features_single(doc,periodo)
    b=a.groupby('MARCA')['MARCA'].transform('count')
    usg.append(pd.concat([a, b], axis=1))
c=pd.concat(usg)



In [ ]:

# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo persona
terminales_doc_1 = []
data_term=terminales[['nro_documento_hash','PERIODO','lanzamiento_equipo']]
for mes in tqdm(data_term['PERIODO'].unique()):
        partial = data_term[data_term['PERIODO']==mes][['PERIODO','nro_documento_hash']] # verano de enero
        q = data_term[(data_term['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                     & (data_term['PERIODO']==mes)]
        
        q1 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).mean().add_prefix('doc_ter_prom_').reset_index().fillna(0)
        q2 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).min().add_prefix('doc_ter_min_').reset_index().fillna(0)
        q3 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).max().add_prefix('doc_ter_max_').reset_index().fillna(0)
        q4 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).std().add_prefix('doc_ter_std_').reset_index().fillna(0)

        

        terminales_doc_1.append(partial.merge(q1,on='nro_documento_hash',how='left')
                                .merge(q2,on='nro_documento_hash',how='left').merge(q3,on='nro_documento_hash',how='left')
                                .merge(q4,on='nro_documento_hash',how='left')
                        )
terminales_doc_1 = pd.concat(terminales_doc_1,sort=False)
terminales_doc_1=terminales_doc_1.drop_duplicates()
terminales_doc_1['key_documento']=terminales_doc_1.PERIODO.astype(str)+'_'+terminales_doc_1.nro_documento_hash.astype(str)

In [ ]:
terminales_doc= []  
hist=terminales[['PERIODO','nro_documento_hash','MARCA','MODELO','NUEVA_GAMMA','OS','DEVICE_TYPE']]
for mes in tqdm(hist['PERIODO'].unique()):
    partial=hist[hist['PERIODO']==mes][['PERIODO','nro_documento_hash']]
    partial=partial.drop_duplicates()
    q = hist[(hist['nro_documento_hash'].isin(partial['nro_documento_hash'].unique())) 
            & (hist['PERIODO']==mes)      
                    ]
    q1=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').nunique().add_suffix('_nunique_prod_').fillna(0)
    q2=q.drop('PERIODO',axis=1).groupby('nro_documento_hash').count().add_suffix('_count_prod_').fillna(0)


    terminales_doc.append(
     partial.merge(q1,on='nro_documento_hash',how='left').merge(q2,on='nro_documento_hash',how='left')
       
    )
terminales_doc = pd.concat(terminales_doc,sort=False).fillna(0)
terminales_doc.drop_duplicates(inplace=True)
terminales_doc['key_documento']=terminales_doc.PERIODO.astype(str)+'_'+terminales_doc.nro_documento_hash.astype(str)

In [ ]:
######################################33
terminales.drop(['PERIODO','nro_telefono_hash','LANZAMIENTO','PERIODO_parsed','nro_documento_hash','aniomes_lanz'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,terminales,on=['key_cliente'],how='left')
print(df_full_hist.shape)

terminales_doc.drop(['PERIODO','nro_documento_hash','MODELO_count_prod_','NUEVA_GAMMA_count_prod_','OS_count_prod_','DEVICE_TYPE_count_prod_'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,terminales_doc,on=['key_documento'],how='left')

terminales_doc_1.drop(['PERIODO','nro_documento_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,terminales_doc_1,on=['key_documento'],how='left')

print(df_full_hist.shape)

# convergente

In [ ]:
convergente=convergente_202201_202204.copy()
convergente['key_documento']=convergente.PERIODO.astype(str)+'_'+convergente.nro_documento_hash.astype(str)

In [ ]:
tiene_prod_total=('TIENE_PROD_1','TIENE_PROD_2','TIENE_PROD_3')
convergente['tiene_prod_total']=convergente.loc[:,tiene_prod_total].sum(axis=1) 

convergente['peso_PROD_1']=convergente['TIENE_PROD_1']/convergente['tiene_prod_total']
convergente['peso_PROD_2']=convergente['TIENE_PROD_2']/convergente['tiene_prod_total']
convergente['peso_PROD_3']=convergente['TIENE_PROD_3']/convergente['tiene_prod_total']

In [ ]:
convergente['GIRO'] = replace_low_classes(convergente['GIRO'],thresh=500,string='Otros').values # 7 =7 a más
convergente['SUBGIRO'] = replace_low_classes(convergente['SUBGIRO'],thresh=500,string='Otros').values # 7 =7 a más

convergente['GIRO']=convergente['GIRO'].astype('category')
convergente['SUBGIRO']=convergente['SUBGIRO'].astype('category')

convergente['GIRO']= convergente['GIRO'].cat.codes
convergente['SUBGIRO']= convergente['SUBGIRO'].cat.codes

In [ ]:
# nro de clientes por Tipo de scort de categoria
convergente['GIRO_CALC']=convergente.GIRO
for mes in tqdm(convergente['PERIODO'].unique()):
    NCLIENTES_GIRO=convergente.loc[(convergente.PERIODO==mes)].GIRO.value_counts()
    convergente.loc[convergente.PERIODO==mes,'GIRO_CALC']=convergente.loc[convergente.PERIODO==mes,'GIRO'].map(NCLIENTES_GIRO)
convergente['GIRO_CALC']=convergente['GIRO_CALC'].astype('int')

convergente['SUBGIRO_CALC']=convergente.SUBGIRO
for mes in tqdm(convergente['PERIODO'].unique()):
    NCLIENTES_SUBGIRO=convergente.loc[(convergente.PERIODO==mes)].SUBGIRO.value_counts()
    convergente.loc[convergente.PERIODO==mes,'SUBGIRO_CALC']=convergente.loc[convergente.PERIODO==mes,'SUBGIRO'].map(NCLIENTES_SUBGIRO)
convergente['SUBGIRO_CALC']=convergente['SUBGIRO_CALC'].astype('int')



In [ ]:

# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo persona
convergente_hist = []
data_conv=convergente[['nro_documento_hash','PERIODO','tiene_prod_total']]
for mes in tqdm(data_conv['PERIODO'].unique()):
        partial = data_conv[data_conv['PERIODO']==mes][['PERIODO','nro_documento_hash']] # verano de enero
        q = data_conv[(data_conv['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                     & (data_conv['PERIODO']<=mes)]
        q_ = data_conv[(data_conv['nro_documento_hash'].isin(partial['nro_documento_hash'].unique()))
                     & (data_conv['PERIODO']==mes)]
        
        q1 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).sum().add_prefix('doc_conv_prom_').reset_index().fillna(0)
        q2 = q_.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).sum().add_prefix('doc_conv_prom2_').reset_index().fillna(0)
        q3 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).mean().add_prefix('doc_conv_mean_').reset_index().fillna(0)
        q4 = q_.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).mean().add_prefix('doc_conv_mean2_').reset_index().fillna(0)
        q5 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).min().add_prefix('doc_conv_min_').reset_index().fillna(0)
        q6 = q_.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).min().add_prefix('doc_conv_min2_').reset_index().fillna(0)
        q7 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).max().add_prefix('doc_conv_max_').reset_index().fillna(0)
        q8 = q_.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).max().add_prefix('doc_conv_max2_').reset_index().fillna(0)
        q9 = q.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).std().add_prefix('doc_conv_std_').reset_index().fillna(0)
        q10 = q_.drop('PERIODO',axis=1).groupby(['nro_documento_hash']).std().add_prefix('doc_conv_std2_').reset_index().fillna(0)
        
        convergente_hist.append(partial.merge(q1,on='nro_documento_hash',how='left')
                                .merge(q2,on='nro_documento_hash',how='left').merge(q3,on='nro_documento_hash',how='left')
                                .merge(q4,on='nro_documento_hash',how='left').merge(q5,on='nro_documento_hash',how='left')
                                .merge(q6,on='nro_documento_hash',how='left').merge(q7,on='nro_documento_hash',how='left')
                                .merge(q8,on='nro_documento_hash',how='left').merge(q9,on='nro_documento_hash',how='left')
                                .merge(q10,on='nro_documento_hash',how='left')
                        )
convergente_hist = pd.concat(convergente_hist,sort=False)
convergente_hist=convergente_hist.drop_duplicates()
convergente_hist['key_documento']=convergente_hist.PERIODO.astype(str)+'_'+convergente_hist.nro_documento_hash.astype(str)

In [ ]:
convergente.drop(['PERIODO','nro_documento_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,convergente,on=['key_documento'],how='left')

convergente_hist.drop(['PERIODO','nro_documento_hash'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,convergente_hist,on=['key_documento'],how='left')

print(df_full_hist.shape)

# roaming

In [ ]:
roaming=roaming_202201_202204.drop_duplicates()
roaming['key_cliente']=roaming.PERIODO.astype(str)+'_'+roaming.nro_telefono_hash.astype(str)
roaming=pd.merge(roaming,df_full_hist[['key_cliente','nro_documento_hash']],on=['key_cliente'],how='left')
roaming['key_documento']=roaming.PERIODO.astype(str)+'_'+roaming.nro_documento_hash.astype(str)
roaming.drop(['nro_telefono_hash','key_cliente'],axis=1,inplace=True)
roaming.shape

In [ ]:
roaming['FECHATRAFICO'] = pd.to_datetime(roaming['FECHATRAFICO'] , format='%d/%m/%Y')

In [ ]:
roaming=roaming[roaming['FECHATRAFICO']>=pd.to_datetime('2022-01-01 00:00:00')]
roaming['codmes_fec_traf']=roaming['FECHATRAFICO'].astype(str).str.slice(0,4)+roaming['FECHATRAFICO'].astype(str).str.slice(5,7)
roaming.shape

In [ ]:
roaming['TIPOSERVICIO'] = replace_low_classes(roaming['TIPOSERVICIO'],thresh=1000,string='TIPO2').values # 7 =7 a más

In [ ]:
# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo persona
roaming_hist = []
data_roaming=roaming[['key_documento','PERIODO','MINUTOS','GIGAS','MENSAJES']]
for mes in tqdm(data_roaming['PERIODO'].unique()):
        partial = data_roaming[data_roaming['PERIODO']==mes][['PERIODO','key_documento']] # verano de enero
        q = data_roaming[(data_roaming['key_documento'].isin(partial['key_documento'].unique()))
                     & (data_roaming['PERIODO']==mes)]
        
        q1 = q.drop('PERIODO',axis=1).groupby(['key_documento']).sum().add_prefix('key_sum_').reset_index().fillna(0)
        q2 = q.drop('PERIODO',axis=1).groupby(['key_documento']).mean().add_prefix('key_prom_').reset_index().fillna(0)
        q3 = q.drop('PERIODO',axis=1).groupby(['key_documento']).median().add_prefix('key_md_').reset_index().fillna(0)
        roaming_hist.append(partial.merge(q1,on='key_documento',how='left')
                                .merge(q2,on='key_documento',how='left').merge(q3,on='key_documento',how='left')
                        )
roaming_hist = pd.concat(roaming_hist,sort=False)
roaming_hist=roaming_hist.drop_duplicates()

In [ ]:
# total de categorias (grupo total) usadas vs scort de Categoria a nivel de codigo persona
roaming_hist2 = []
for mes in tqdm(roaming['PERIODO'].unique()):
        partial = roaming[roaming['PERIODO']==mes][['PERIODO','key_documento']] # verano de enero
        q = roaming[(roaming['key_documento'].isin(partial['key_documento'].unique()))
                           & (roaming.PERIODO==mes)]
        
        q1 = q.groupby(['key_documento','TIPOSERVICIO'])['MINUTOS'].sum().unstack().add_prefix('sum_min_').reset_index().fillna(0)
        q2 = q.groupby(['key_documento','TIPOSERVICIO'])['MINUTOS'].mean().unstack().add_prefix('mean_min_').reset_index().fillna(0)
        q3 = q.groupby(['key_documento','TIPOSERVICIO'])['GIGAS'].sum().unstack().add_prefix('sum_gigas_').reset_index().fillna(0)
        q4 = q.groupby(['key_documento','TIPOSERVICIO'])['GIGAS'].mean().unstack().add_prefix('mean_gigas_').reset_index().fillna(0)
        q5 = q.groupby(['key_documento','TIPOSERVICIO'])['MENSAJES'].sum().unstack().add_prefix('sum_sms_').reset_index().fillna(0)
        q6 = q.groupby(['key_documento','TIPOSERVICIO'])['MENSAJES'].mean().unstack().add_prefix('mean_sms_').reset_index().fillna(0)

        roaming_hist2.append(partial.merge(q1,on='key_documento',how='left')
                           .merge(q2,on='key_documento',how='left').merge(q3,on='key_documento',how='left')
                           .merge(q4,on='key_documento',how='left').merge(q5,on='key_documento',how='left')
                             .merge(q6,on='key_documento',how='left')
                        )
roaming_hist2 = pd.concat(roaming_hist2,sort=False)
roaming_hist2=roaming_hist2.drop_duplicates()

In [ ]:
roaming_hist.drop(['PERIODO'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,roaming_hist,on=['key_documento'],how='left')
df_full_hist.shape

roaming_hist2.drop(['PERIODO'],axis=1,inplace=True)
df_full_hist=pd.merge(df_full_hist,roaming_hist2,on=['key_documento'],how='left')
df_full_hist.shape


# INGRESO DE VARIABLES AL MODELO

In [ ]:
#df_full_hist.to_csv('df_full_hist_2.csv',sep=',')

df_full_hist=pd.read_csv(file_path+'df_full_hist_2.csv')
df_full_hist_2=pd.read_csv(file_path+'df_full_hist.csv')
perfil_digital=pd.read_csv(file_path+'perfil_digital.csv')
suscriptora=pd.read_csv(file_path+'suscriptora.csv')

#df_full_hist.drop(['Unnamed: 0'],axis=1,inplace=True)
tmp=df_full_hist_2[['key_cliente','marca_counts','modelo_counts','ngama_counts'
                                        ,'os_counts','device_counts','rel_use','rel_use_full']]
df_full_hist=pd.merge(df_full_hist,tmp,on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,perfil_digital[['key_cliente','scorecat_counts']],on=['key_cliente'],how='left')
df_full_hist=pd.merge(df_full_hist,suscriptora[['key_cliente','tipo_adq_counts']],on=['key_cliente'],how='left')
del df_full_hist_2,perfil_digital,suscriptora
df_full_hist.shape


In [ ]:
df=df_full_hist.copy()
df.drop(['NUMPERIODO','nro_documento_hash', 'nro_telefono_hash', 'key_documento'],axis=1,inplace=True)
categorical=[
# suscriptora
        'TIPO_ADQ'
    #terminales
        #,'MARCA','MODELO','NUEVA_GAMMA','OS','DEVICE_TYPE'
                ]

In [ ]:
def datos_imputados(variable):
    # buscamos variables categoricas y discretas
    temp = df.groupby([variable])[variable].count()/np.float64(len(df))
    frecuencia_cat = [x for x in temp.loc[temp>0.01].index.values]
    
    df[variable] = np.where(df[variable].isin(frecuencia_cat), df[variable], 'Raro')

for var in categorical:
    datos_imputados(var)

In [ ]:
def codificar_var_categoricas(var, target):
        # tomamos como referencia el target para poder codificar
        ordenamos_cat = df.groupby([var])[target].sum().sort_values().index # ordenamos en funcion a peso del target
        transf_cat = {k:i for i, k in enumerate(ordenamos_cat, 0)} # transformamos a continuas
        
        # codificamos el set
        df[var] = df[var].map(transf_cat)

# codificamos las variables
for var in categorical:
    codificar_var_categoricas(var, 'TARGET')

In [ ]:
df_forw=df.copy()
df_forw['clase'] = 0
df_forw.loc[df_forw.TARGET.notnull(),'clase']='train' #training
df_forw.loc[df_forw.TARGET.isnull(),'clase']='test' #back_test
train=df_forw.loc[df_forw.clase=='train'].set_index('key_cliente').drop(['clase','TARGET'],axis=1)
test=df_forw.loc[df_forw.clase=='test'].set_index('key_cliente').drop(['clase','TARGET'],axis=1)

y_train=df_forw.loc[df_forw.clase=='train'].set_index('key_cliente').TARGET.astype(int)  ## cambiamos el tipo de target
y_test=df_forw.loc[df_forw.clase=='test'].set_index('key_cliente').TARGET ## cambiamos el tipo de target
# limpiamos nommbres y caracteres extraños de nombres de columnas
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
print('Training',train.shape,'Testing',test.shape) ###(358487, 2630) (396666, 2630)

In [ ]:
# eliminamos valores unicos

cols_with_onlyone_val = train.columns[train.nunique() == 1]
print(cols_with_onlyone_val)

print(train.shape)
print(test.shape)

train.drop(cols_with_onlyone_val.values, axis=1, inplace=True)
test.drop(cols_with_onlyone_val.values, axis=1, inplace=True)

print(train.shape)
print(test.shape)

NUM_OF_DECIMALS = 32
train = train.round(NUM_OF_DECIMALS)
test = test.round(NUM_OF_DECIMALS)

print(train.shape)
print(test.shape)

In [ ]:
colsToRemove = []
columns = train.columns
for i in range(len(columns)-1):
    v = train[columns[i]].values
    dupCols = []
    for j in range(i + 1,len(columns)):
        if np.array_equal(v, train[columns[j]].values):
            colsToRemove.append(columns[j])
train.drop(colsToRemove, axis=1, inplace=True) 
test.drop(colsToRemove, axis=1, inplace=True) 
train.shape

In [ ]:
missing_train = missing_values_table(train)
missing_train_vars = list(missing_train.index[missing_train['% de valores totales'] > 90])
missing_test = missing_values_table(test)
missing_test_vars = list(missing_test.index[missing_test['% de valores totales'] > 90])
missing_columns = list(set(missing_test_vars + missing_train_vars))
# Drop the missing columns
train = train.drop(columns = missing_columns)
test = test.drop(columns = missing_columns)

from sklearn import random_projection

weight = ((train != 0).sum()/len(train)).values
tmp_train = train[train!=0]
tmp_test = test[test!=0]
tmp = pd.concat([train,test])#RandomProjection
ntrain = len(train)
ntest = len(test)

train["weight_count"] = (tmp_train*weight).sum(axis=1)
test["weight_count"] = (tmp_test*weight).sum(axis=1)

train["count_not0"] = (train != 0).sum(axis=1)
test["count_not0"] = (test != 0).sum(axis=1)

train["sum"] = train.sum(axis=1)
test["sum"] = test.sum(axis=1)

train["var"] = tmp_train.var(axis=1)
test["var"] = tmp_test.var(axis=1)

train["mean"] = tmp_train.mean(axis=1)
test["mean"] = tmp_test.mean(axis=1)

train["std"] = tmp_train.std(axis=1)
test["std"] = tmp_test.std(axis=1)

train["max"] = tmp_train.max(axis=1)
test["max"] = tmp_test.max(axis=1)

train["min"] = tmp_train.min(axis=1)
test["min"] = tmp_test.min(axis=1)
#######################
#train["median"] = tmp_train.median(axis=1)
#test["median"] = tmp_test.median(axis=1)

#train["skew"] = tmp_train.skew(axis=1)
#test["skew"] = tmp_test.skew(axis=1)

#train["kurtosis"] = tmp_train.kurtosis(axis=1)
#test["kurtosis"] = tmp_test.kurtosis(axis=1)

del(tmp_train)
del(tmp_test)


from scipy.stats import ks_2samp
THRESHOLD_P_VALUE = 0.01 #need tuned
THRESHOLD_STATISTIC = 0.3 #need tuned
diff_cols = []
for col in train.columns:
    statistic, pvalue = ks_2samp(train[col].values, test[col].values)
    if pvalue <= THRESHOLD_P_VALUE and np.abs(statistic) > THRESHOLD_STATISTIC:
        diff_cols.append(col)
for col in diff_cols:
    if col in train.columns:
        train.drop(col, axis=1, inplace=True)
        test.drop(col, axis=1, inplace=True)
train.shape

test['full_count_null'] = test.apply(lambda x: x.count(), axis=1)
train['full_count_null'] = train.apply(lambda x: x.count(), axis=1)

In [ ]:
print('Training',train.shape,'Testing',test.shape) ###(358487, 2630) (396666, 2630)

#### min_child_samples  : Número mínimo de datos necesarios en un niño (hoja). Según los documentos de LightGBM, este es un parámetro muy importante para evitar el sobreajuste.

#### num_leaves (LightGBM): máximo de hojas de árboles para los estudiantes básicos. Un valor más alto da como resultado árboles más profundos

# modelo de lightgbm

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= -1 # -1
                            ,max_bin = 8
                            ,min_child_samples=100 #100
                            ,num_leaves=10  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            #,learning_rate=0.1
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs1 = []
train_probs1 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= 8 # -1
                            ,max_bin = 8
                            ,min_child_samples=60 #100
                            ,num_leaves=20  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            #,learning_rate=0.1
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs1.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs1.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs1 = pd.concat(test_probs1, axis=1).mean(axis=1)
train_probs1 = pd.concat(train_probs1)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs1.loc[y_train.index]))

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs2 = []
train_probs2 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= -1 # -1
                            ,max_bin = 8
                            ,min_child_samples=50 #100
                            ,num_leaves=20  #20
                            #,reg_alpha=0 # 0
                            #,reg_lambda=1 # 1
                            #,learning_rate=0.1
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs2.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs2.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs2 = pd.concat(test_probs2, axis=1).mean(axis=1)
train_probs2 = pd.concat(train_probs2)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs2.loc[y_train.index]))

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs3 = []
train_probs3 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= -1 # -1
                            ,max_bin = 8
                            ,min_child_samples=100 #100
                            ,num_leaves=10  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            ,learning_rate=0.05
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs3.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs3.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs3 = pd.concat(test_probs3, axis=1).mean(axis=1)
train_probs3 = pd.concat(train_probs3)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs3.loc[y_train.index]))

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs4 = []
train_probs4 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            ,class_weight='balanced'
                            ,max_depth= -1 # -1
                            ,max_bin = 8
                            ,min_child_samples=100 #100
                            ,num_leaves=10  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            #,learning_rate=0.1
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs4.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs4.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs4 = pd.concat(test_probs4, axis=1).mean(axis=1)
train_probs4 = pd.concat(train_probs4)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs4.loc[y_train.index]))

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs5 = []
train_probs5 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= 5 # -1
                            ,max_bin = 8
                            ,min_child_samples=100 #100
                            ,num_leaves=10  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            ,learning_rate=0.05
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs5.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs5.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs5 = pd.concat(test_probs5, axis=1).mean(axis=1)
train_probs5 = pd.concat(train_probs5)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs5.loc[y_train.index]))

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs6 = []
train_probs6 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= -1 # -1
                            ,max_bin = 8
                            ,min_child_samples=50 #100
                            ,num_leaves=10  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            #,learning_rate=0.1
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=20, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs6.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs6.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs6 = pd.concat(test_probs6, axis=1).mean(axis=1)
train_probs6 = pd.concat(train_probs6)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs6.loc[y_train.index]))

In [ ]:
test_envio=test_probs6.copy()
test_envio.name = "TARGET"
test_envio=pd.DataFrame(test_envio).reset_index()
#test_envio['nro_telefono_hash']=0
test_envio['nro_telefono_hash']=test_envio.key_cliente.str.slice(7,500)
#test_envio=pd.merge(test_envio,df_full_hist[['key_cliente','nro_telefono_hash']],on=['key_cliente'],how='left')
test_envio.drop('key_cliente', axis=1,inplace=True)
summission=test_envio[['nro_telefono_hash','TARGET']].set_index('nro_telefono_hash')
summission.to_csv('summission_T6.csv',sep=',')

In [ ]:
# usamos el algoritms para poder entrenar y predecir
gc.collect()
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs7 = []
train_probs7 = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(
                            n_estimators=10000
                            ,importance_type='gain'
                            #,class_weight='balanced'
                            ,max_depth= -1 # -1
                            #,max_bin = 8
                            #,min_child_samples=100 #100
                            ,num_leaves=10  #20
                            ,reg_alpha=0 # 0
                            ,reg_lambda=1 # 1
                            #,learning_rate=0.1
                                )
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) #25 despues  del tuning
    test_probs7.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs7.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs7 = pd.concat(test_probs7, axis=1).mean(axis=1)
train_probs7 = pd.concat(train_probs7)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs7.loc[y_train.index]))

# emsamble promedio

In [ ]:
ensembled_probs1=(test_probs + test_probs1 + test_probs2 + test_probs3 +test_probs4+test_probs5+ test_probs6 + test_probs7 )/8
#print(test_probs_apv.shape,test_probs.shape)

test_envio=ensembled_probs1.copy()
test_envio.name = "TARGET"
test_envio=pd.DataFrame(test_envio).reset_index()
#test_envio['nro_telefono_hash']=0
test_envio['nro_telefono_hash']=test_envio.key_cliente.str.slice(7,500)
#test_envio=pd.merge(test_envio,df_full_hist[['key_cliente','nro_telefono_hash']],on=['key_cliente'],how='left')
test_envio.drop('key_cliente', axis=1,inplace=True)
summission=test_envio[['nro_telefono_hash','TARGET']].set_index('nro_telefono_hash')
summission.to_csv('summission_lgbms_prom.csv',sep=',')